In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [2]:
data = pd.read_excel('test_monster/DataSet.xlsx')

In [3]:
def get_receipt(receipt):
    return receipt.split()[1]

In [4]:
def get_date(receipt):
    return pd.to_datetime(receipt.split()[3], format='%d.%m.%Y')

In [5]:
def prepare_dataset(df):
    df['receipt'] = df['Чек'].apply(get_receipt)
    df['date'] = df['Чек'].apply(get_date)

In [6]:
prepare_dataset(data)

### Создадим датасет с уникальными клиентами, которых будем сегментировать по группам

In [7]:
clients = pd.DataFrame({'client': data['Карта лояльности'].unique()})

In [8]:
clients.dropna(inplace=True)

### Разделим клиентов на группы по дате последнего посещения, количеству посещений и количеству потраченных денег  
Разделение на группы было осуществлено так, чтобы лучших клиентов оказалось как можно меньше, а худших - больше.

In [9]:
last_date = data['date'].max()

In [10]:
last_date

Timestamp('2018-12-31 00:00:00')

In [11]:
clients['last_visit_days'] = 0
clients['last_visit_group'] = 0
clients['transactions_sum'] = 0
clients['transactions_group'] = 0
clients['purchase_sum'] = 0
clients['purchase_group'] = 0

In [12]:
for name, group in tqdm(data.groupby('Карта лояльности')):
    clients.loc[clients['client'] == name, 'last_visit_days'] = (last_date - group['date'].max()).days
    clients.loc[clients['client'] == name, 'transactions_sum'] = group['receipt'].unique().shape[0]
    clients.loc[clients['client'] == name, 'purchase_sum'] = group['Сумма Итоговая'].sum()

In [13]:
# days_33 = clients['last_visit_days'].quantile(.33)
# days_66 = clients['last_visit_days'].quantile(.66)
# transactions_33 = clients['transactions_sum'].quantile(.33)
# transactions_66 = clients['transactions_sum'].quantile(.66)
# purchase_33 = clients['purchase_sum'].quantile(.33)
# purchase_66 = clients['purchase_sum'].quantile(.66)
# 7 30
# 1 4
# 7000 1500

In [14]:
def get_last_visit_group(days):
    if days <= 6:
        return 1
    elif days > 6 and days <= 30:
        return 2
    else:
        return 3

In [15]:
def get_transactions_group(transactions_sum):
    if transactions_sum == 1:
        return 3
    elif transactions_sum > 1 and transactions_sum <= 4:
        return 2
    else:
        return 1

In [16]:
def get_purchase_group(purchase_sum):
    if purchase_sum > 7000:
        return 1
    elif purchase_sum <= 7000 and purchase_sum > 1500:
        return 2
    else:
        return 3

In [17]:
clients['last_visit_group'] = clients['last_visit_days'].apply(get_last_visit_group)
clients['transactions_group'] = clients['transactions_sum'].apply(get_transactions_group)
clients['purchase_group'] = clients['purchase_sum'].apply(get_purchase_group)

In [18]:
clients['last_visit_group'].value_counts()

3    4698
2    3446
1    2264
Name: last_visit_group, dtype: int64

In [19]:
clients['transactions_group'].value_counts()

3    5035
2    3129
1    2244
Name: transactions_group, dtype: int64

In [20]:
clients['purchase_group'].value_counts()

3    4439
2    3868
1    2101
Name: purchase_group, dtype: int64

### После такого разбиения можно немного проанализировать полученные группы клиентов

In [21]:
def get_rfm_segment(segment):
    res = 0
    for rfm in segment:
         res += clients.loc[(clients['last_visit_group'] == rfm[0]) & (clients['transactions_group'] == rfm[1]) & (clients['purchase_group'] == rfm[2])].shape[0] * 100 / clients.shape[0]
    return res

In [63]:
ideal = [[1, 1, 1]]
loyal = [[1, 1, 2], [1, 1, 3]]
perpective = [[1, 2, 1], [1, 2, 2]]
new = [[1, 3, 1], [1, 3, 2]]
sleep_loyal = [[2, 1, 1], [2, 1, 2]]
sleep_perspective = [[2, 2, 1], [2, 2, 2]]
ex_loyal = [[3, 1, 1], [3, 1, 2]]
ex_perspective = [[3, 2, 1], [3, 2, 2]]
stray = [[2, 3, 1], [2, 3, 2], [2, 3, 3]]
lost = [[3, 3, 1], [3, 3, 2], [3, 3, 3]]
low_check = [[1, 2, 3], [1, 3, 3], [2, 1, 3], [2, 2, 3], [3, 1, 3], [3, 2, 3]]

In [64]:
get_rfm_segment(ideal)

8.358954650269023

Видно, что в результате сегментации было выделено 8.4% идеальных клиентов.  
Это те клиенты, которые посещают магазин часто, тратят много, а в последний раз заходили в течение недели.  
Нет никакой необходимости тратить бюджет на скидки или акционные предложения для данной группы клиентов, но важно удержать их и мотивировать на дальнейшие покупки. Неназойливые знаки внимания, дающие понять, что клиент важен для компании. Но не стоит их утомлять лишними коммуникациями.  
Анализ данного сегмента может указать путь к построению подобных отношений с другими клиентами, если, например, попросить их оставить отзыв о компании.

In [65]:
get_rfm_segment(loyal)

2.9496541122213684

Почти 3% - лояльные клиенты с низким и средним чеком.

In [66]:
get_rfm_segment(perspective)

4.4965411222136815

4.5% составляют перспективные клиенты - те, кто приходил недавно и делает регулярно несколько крупных или средних покупок.  
Важно удержать их интерес. Данная группа клиентов имеет все шансы перейти в категорию постоянных клиентов. Стоит узнать их мнение о компании, уделить им внимание, но быть аккуратными со скидками - данный сегмент покупателей имеет высокий шанс стать клиентами за полную стоимость.

In [67]:
get_rfm_segment(new)

1.5949269792467333

1.6% составили новички со средним и высоким чеком.  
Важно наладить с ними контакт, произвести хорошее впечатление после первого закомства, что поможет перевести их в категорию лояльных.

In [68]:
get_rfm_segment(sleep_perspective)

8.656802459646427

8.7% - спящие перспективные клиенты.  
Данные клиенты посещали магазин в последий месяц, заходили довольно часто и тратили много. Онако, они находятся под угрозой оттока.  Они еще помнят о компании, стоит привлечь их внимание. Это может быть хорошая персональная скидка, рекомендации и прочее. Цель - вернуть клиента в стан лояльных или хотя бы перспективных.

In [69]:
get_rfm_segment(sleep_loyal)

7.926594926979247

8.2% - спящие лояльные клиенты

In [70]:
get_rfm_segment(ex_loyal)

1.911990776325903

2.1% попадают под категорию бывших лояльных клиентов.  
Данный сегмент делал покупки много и часто, однако, уже больше месяца не заходил в магазин. Возможно, клиент переехал, и поблизости не оказалось магазина данной сети. Но если он просто начал ходит в другой - его можно вернуть. Подойдут те же бонусы, что и для предыдущей категории, возможно даже более весомые.

In [71]:
get_rfm_segment(ex_perspective)

8.137970791698693

12.2% - бывшие перспективные

In [72]:
get_rfm_segment(stray)

12.893927747886242

12.9% клиентов оказались в магазине случайно в прошлом месяце, сделали одну покупку.  
Возможно, проходили мимо, или их привлек какой-то определенный продукт. Это можно проанализировать. Есть вероятность привлечь клиента, но не стоит тратить на это много ресурсов.

In [73]:
get_rfm_segment(lost)

30.870484242890086

30.9% клиентов можно отнести к категории потерянных.  
Похоже на прошлую категорию, только были в магазине больше месяца назад. Данный сегмент еще менее вероятно станет постоянным клиентом сети.

In [74]:
get_rfm_segment(low_check)

12.202152190622598

### Для каждого клиента посчитаем сколько дней он является клиентом и средний чек покупок в день

In [75]:
clients['days_client'] = 0

In [76]:
for name, group in tqdm(data.groupby('Карта лояльности')):
    clients.loc[clients['client'] == name, 'days_client'] = (last_date - group['date'].min()).days + 1

In [77]:
# Средний размер покупок за отчетный период
clients['check_average'] = clients['purchase_sum'] / 92

### Посчитаем LTV для каждого клиента и посмотрим средний LTV по категориям

In [78]:
clients['LTV'] = clients['days_client'] * clients['check_average']

In [79]:
def get_ltv_rfm_segment(segment):
    ltv = 0
    size = np.array(segment).shape[0]
    for rfm in segment:
        ltv += clients.loc[(clients['last_visit_group'] == rfm[0]) & (clients['transactions_group'] == rfm[1]) & (clients['purchase_group'] == rfm[2]), 'LTV'].mean()
    return ltv / size

In [80]:
get_ltv_rfm_segment(ideal)

25479.313830584728

В категории идеальных клиентов ожидаемо самый высокий LTV.  
Как говорилось ранее, эти клиенты и так лояльны, данной категории не нужны скидки. Но необходимо удержать их.

In [81]:
get_ltv_rfm_segment(loyal)

1791.7903476989575

Лояльные клиенты с низким и средним чеком. За счет небольших сумм покупок LTV получился низким. Стоит простимулировать клиентов покупать больше, предлагая, например, сопутствующие товары.

In [82]:
get_ltv_rfm_segment(perspective)

4031.1689478414205

Хороший LTV у перспективных клиентов. Необходимо перевести их в категорию лояльных, проявить к ним внимание.  
Они уже не раз бывали в магазине, заходили недавно, тратили немаленькие суммы. Они готовы, стоит их лишь подогреть - например, диалогом.

In [83]:
get_ltv_rfm_segment(new)

329.3031744734507

Новички. Низкий LTV за счет своего недавнего появления. Необходимо простимулировать к повторным покупкам. Наладить диалог. Сообщить о поступлении товаров, которыми клиент интересовался.

In [84]:
get_ltv_rfm_segment(sleep_loyal), get_ltv_rfm_segment(sleep_perspective)

(10721.716528132993, 4720.4973822222455)

Под угрозой оттока. Вернуть! Хорошие персональные скидки и рекомендации - сюда.

In [85]:
get_ltv_rfm_segment(ex_loyal), get_ltv_rfm_segment(ex_perspective)

(9475.28796358808, 5860.4673503973845)

Бывшие лояльные. То же самое, что и с предыдущими.

In [86]:
get_ltv_rfm_segment(stray)

1047.0212803599165

Залетные. Можно попробовать.

In [87]:
get_ltv_rfm_segment(lost)

3095.098322772852

Потерянные. Попробовать вернуть.

In [88]:
clients['LTV'].mean()

4497.368261872129

### Посчитаем сумму скидок на каждого клиента, среднее количество покупок за посещение,  топ товаров, предпочитают ли own trademark

In [89]:
clients['discount_prop'] = 0
clients['purchases_mean'] = 0
clients['own_trademark_prop'] = 0

In [90]:
data['own_trademark'] = data['Название товара'].apply(lambda x: 1 if 'PL' in x else 0)

In [91]:
for name, group in tqdm(data.groupby('Карта лояльности')):
    clients.loc[clients['client'] == name, 'discount_prop'] = group['Сумма скидки'].sum() * 100 / (group['Цена'] * group['Количество']).sum()
    clients.loc[clients['client'] == name, 'purchases_mean'] = group['Количество'].sum()
    clients.loc[clients['client'] == name, 'own_trademark_prop'] = group[group['own_trademark'] == 1]['Количество'].sum()
clients['own_trademark_prop'] = clients['own_trademark_prop'] * 100 / clients['purchases_mean']
clients['purchases_mean'] = clients['purchases_mean'] / clients['transactions_sum']

In [92]:
def get_segment_info(R, F, M):
    return (clients.loc[(clients['last_visit_group'] == R) & (clients['transactions_group'] == F) & (clients['purchase_group'] == M), 'discount_prop'].mean(), 
           clients.loc[(clients['last_visit_group'] == R) & (clients['transactions_group'] == F) & (clients['purchase_group'] == M), 'own_trademark_prop'].mean(),
           clients.loc[(clients['last_visit_group'] == R) & (clients['transactions_group'] == F) & (clients['purchase_group'] == M), 'purchases_mean'].mean())

In [93]:
def get_average_info(segment):
    info = np.array([.0, .0, .0])
    size = np.array(segment).shape[0]
    for rfm in segment:
        info += np.array(get_segment_info(rfm[0], rfm[1], rfm[2]))
    return info / size

In [94]:
get_average_info(ideal)

array([ 8.79382097, 35.93968161,  8.23919183])

In [95]:
get_average_info(loyal)

array([24.96534719, 40.55678613,  2.90201899])

In [96]:
get_average_info(perspective)

array([ 7.44472845, 26.12358985,  8.9491787 ])

In [97]:
get_average_info(new)

array([ 6.72950458, 20.78270875, 20.7741382 ])

In [98]:
get_average_info(sleep_loyal), get_average_info(sleep_perspective)

(array([ 8.73378358, 34.42067181,  5.97014126]),
 array([ 6.50192908, 25.09983143,  9.32594323]))

In [99]:
get_average_info(ex_loyal), get_average_info(ex_perspective)

(array([ 7.16496296, 35.33811183,  5.82450656]),
 array([ 6.07313209, 29.03711411,  9.86940454]))

In [100]:
get_average_info(stray)

array([ 5.55338803, 21.08693221, 14.32752782])

In [101]:
get_average_info(lost)

array([ 4.89588205, 21.35460372, 13.11562149])

In [102]:
get_ltv_rfm_segment(low_check)

529.0550842939465

In [103]:
get_average_info(low_check)

array([15.33493015, 34.97285973,  2.72610593])